In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import torch
import magic

In [ ]:
def pearson_correlation(A, B, d):
    # Calculate mean along the columns (axis=0)
    mean_A = torch.mean(A, dim=0)
    mean_B = torch.mean(B, dim=0)

    # Calculate covariance
    cov_AB = torch.sum((A - mean_A) * (B - mean_B), dim=d)

    # Calculate standard deviation
    std_A = torch.sqrt(torch.sum((A - mean_A) ** 2, dim=d))
    std_B = torch.sqrt(torch.sum((B - mean_B) ** 2, dim=d))

    # Calculate Pearson correlation coefficient
    correlation = cov_AB / (std_A * std_B + 1e-12)

    return correlation


In [40]:
name="p21"

In [41]:
# rna=sc.read_h5ad(f"/project/zduren/durenlab/idebnat/palmetto/Modest/Chrom_spatial/Modest_klnmf/rna{name}.h5ad")
# atac=sc.read_h5ad(f"/project/zduren/durenlab/idebnat/palmetto/Modest/Chrom_spatial/Modest_klnmf/atac{name}.h5ad")
# sprna=sc.read_h5ad(f"/project/zduren/durenlab/idebnat/palmetto/Modest/Chrom_spatial/Modest_klnmf/sprna{name}.h5ad")
# spatac=sc.read_h5ad(f"/project/zduren/durenlab/idebnat/palmetto/Modest/Chrom_spatial/Modest_klnmf/spatac{name}.h5ad")

rna=sc.read_h5ad(f"rna_{name}.h5ad")
atac=sc.read_h5ad(f"atac_{name}.h5ad")
sprna=sc.read_h5ad(f"sprna_{name}.h5ad")
spatac=sc.read_h5ad(f"spatac_{name}.h5ad")

In [42]:
cca=pd.read_csv(f"cca_{name}.tsv", index_col=0, delimiter=" ")
print(cca.shape)

(33459, 20)


In [43]:
train_mu=torch.tensor(cca[cca.index.isin(rna.obs.index)].values)
train_sp=torch.tensor(cca[cca.index.isin(sprna.obs.index)].values)

In [45]:
test_mu=torch.tensor(atac.X.A)

In [46]:
sim=torch.matmul(train_mu, train_sp.t())
sim_avg = sim - torch.mean(sim, dim=0)

wmat=torch.zeros_like(sim_avg)
for i in range(sim_avg.shape[0]):
    values, indices = torch.topk(sim_avg[i, :], k=15, dim=0)

    wmat[i, indices] = 1/15

predd4=torch.matmul(test_mu.t().to(dtype=torch.float), wmat.to(dtype=torch.float))

In [38]:
torch.save(predd4.t(), f'cca_pred_{name}.pt')